In [1]:
import os as os
import requests as req
import zipfile as zipfile
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def data_directories():
    """
    Funcao de criacao das pastas para armazenamento dos dados, caso nao existam.
    
    """

    # Diretorios para armazenamento dos dados coletados
    directories = ['..\\data\\01-collected'
                  ,'..\\data\\02-cleaned'
                  ,'..\\data\\03-organized'
                  ,'..\\data\\04-standardized']
    
    # Log: Mensagem de inicio da criacao dos diretorios
    print(str(datetime.now()) + ': Criacao dos diretorios para armazenamento dos dados iniciada.')
    
    # Para cada diretorio na lista de diretorios
    for directory in directories:
        
        # Verifica se o diretorio existe
        if not os.path.exists(directory):
            
            # Cria o diretorio
            os.makedirs(directory)
            
            # Log: Mensagem de criacao do diretorio
            print(str(datetime.now()) + ': Diretorio ' + directory + ' criado.')
        
        else:
            
            # Log: Mensagem de diretorio existente
            print(str(datetime.now()) + ': Diretorio ' + directory + ' ja existe.')
    
    # Log: Mensagem de fim da criacao dos diretorios
    print(str(datetime.now()) + ': Criacao dos diretorios para armazenamento dos dados finalizada.')

In [3]:
def download_file(**kwargs):
    """
    Funcao que realiza o download do arquivo. Ao final do download, e verificado
    se o arquivo solicitado esta vazio. Caso esteja vazio, o arquivo e apagado. Caso contrario,
    o arquivo e descompactado.
    
    Args:
        url_query (str): URL parametrizada para download do arquivo.
        download_path (str): Caminho do diretorio onde sera salvo o arquivo.
        download_directory (str): Caminho do diretorio para decompactacao do arquivo
    """
    
    # Log: Mensagem de inicio do download do arquivo
    print(str(datetime.now()) + ': Inicio do download do arquivo ' + kwargs['download_path'])
    
    # Prepara a requisicao de download do arquivo
    res = req.get(kwargs['url_query'], stream = True)
    
    # Se for encontrada a URL para download
    if res.status_code == 200:
        
        # Faz o download do arquivo
        with open(kwargs['download_path'], 'wb') as f:
            
            # Log: Mensagem de URL encontrada e de inicio do download
            print(str(datetime.now()) + ': URL encontrada, fazendo o download do arquivo: ' + kwargs['download_path'])
            
            # Download do arquivo em pedacos
            for chunk in res.iter_content(chunk_size = 1024):
                f.write(chunk)
        
        # Verifica o tamanho do arquivo recebido
        file_size = os.stat(kwargs['download_path']).st_size
        
        # Se o tamanho do arquivo for muito 
        # pequeno remove o arquivo da pasta 
        if file_size <= 100:
            
            # Log: Mensagem de remocao por arquivo por nao conter dados
            print(str(datetime.now()) + ': Arquivo ' + kwargs['download_path'] + ' removido por conter poucos dados.')
            
            # Remove o arquivo sem dados
            os.remove(kwargs['download_path'])
            
        # Caso contrario, descompacta o arquivo csv 
        # na pasta e remove o arquivo compactado
        else:
            
            # Log: Mensagem de inicio da descompactacao do arquivo zip
            print(str(datetime.now()) + ': Descompactando o arquivo ' + kwargs['download_path'])
            
            # Descompacta o arquivo zip mantem apenas o csv
            zip_ref = zipfile.ZipFile(kwargs['download_path'], 'r')
            zip_ref.extractall(path=kwargs['download_directory'])
            zip_ref.close()
            os.remove(kwargs['download_path'])
            
            # log: Mensagem de finalizacao do download do arquivo
            print(str(datetime.now()) + ': Fim do download do arquivo ' + kwargs['download_path'])

In [4]:
def collect_data(**kwargs):
    """
    Coleta os dados para cada mes de referencia dentro das datas de inicio e fim informadas.
    
    Args:
        dt_ini_ref (date): Data inicial para coleta dos dados
        dt_fim_ref (date): Data final para coleta dos dados
    """
    
    # Cria os diretorios para armazenamento dos dados
    data_directories()
    
    # Define a data de referencia inicial
    kwargs['dt_ref'] = kwargs['dt_ini_ref']
    
    # Define o diretorio para download dos arquivos
    kwargs['download_directory'] = '..\\data\\01-collected\\'
    
    # Define a URL base de busca dos arquivos para download
    kwargs['download_url'] = 'http://www.portaltransparencia.gov.br/download-de-dados/compras/'
    
    # Define o nome e extensao do arquivo para download
    kwargs['file_name'] = '_Compras'
    kwargs['file_ext'] = '.zip'
    
    # Log: Mensagem de inicio do processo 
    print(str(datetime.now()) + ': Coleta dos arquivos iniciada.')
       
    # Para cada mes de referencia dentro das datas de inicio e fim
    while kwargs['dt_ref'] <= kwargs['dt_fim_ref']:
        
        # Prepara as variaveis do mes de referencia
        kwargs['file_date_ref'] = kwargs['dt_ref'].strftime('%Y%m')
        kwargs['download_file'] = kwargs['file_date_ref'] + kwargs['file_name'] + kwargs['file_ext']
        
        # Define a URL completa para download do arquivo
        kwargs['url_query'] = kwargs['download_url'] + kwargs['file_date_ref']
        
        # Define o caminho completo para armazenamento do arquivo de download
        kwargs['download_path'] = kwargs['download_directory'] + kwargs['download_file']
        
        # Inicia o download do arquivo
        download_file(**kwargs)
        
        # Atualiza o mes de refencia
        kwargs['dt_ref'] += relativedelta(months=1)

    # Log: Mensagem de finalizacao do processo 
    print(str(datetime.now()) + ': Coleta dos arquivos finalizada.')

In [7]:
def main():
    """
    Coleta os dados de Contratos do Governo Federal (Contratos). 
    Origem: Portal da Transparencia
    """
    
    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Datas de Inicio e Fim (YYYY-MM-DD) 
    # Obs.: Dados disponiveis a partir de 2013-01-01
    kwargs['dt_ini_ref'] = datetime.strptime('2013-01-01', "%Y-%m-%d")
    kwargs['dt_fim_ref'] = datetime.strptime('2019-06-01', "%Y-%m-%d")
    
    # Coleta os arquivos
    collect_data(**kwargs)

if __name__ == '__main__':
    main()

2019-09-03 19:08:31.634786: Criacao dos diretorios para armazenamento dos dados iniciada.
2019-09-03 19:08:31.635750: Diretorio ..\data\01-collected ja existe.
2019-09-03 19:08:31.635750: Diretorio ..\data\02-cleaned ja existe.
2019-09-03 19:08:31.635750: Diretorio ..\data\03-organized ja existe.
2019-09-03 19:08:31.635750: Diretorio ..\data\04-standardized ja existe.
2019-09-03 19:08:31.635750: Criacao dos diretorios para armazenamento dos dados finalizada.
2019-09-03 19:08:31.635750: Coleta dos arquivos iniciada.
2019-09-03 19:08:31.635750: Inicio do download do arquivo ..\data\01-collected\201301_Compras.zip
2019-09-03 19:08:37.032093: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\201301_Compras.zip
2019-09-03 19:08:37.718271: Descompactando o arquivo ..\data\01-collected\201301_Compras.zip
2019-09-03 19:08:37.779140: Fim do download do arquivo ..\data\01-collected\201301_Compras.zip
2019-09-03 19:08:37.779140: Inicio do download do arquivo ..\data\01-collected

2019-09-03 19:10:13.317416: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\201408_Compras.zip
2019-09-03 19:10:14.008154: Descompactando o arquivo ..\data\01-collected\201408_Compras.zip
2019-09-03 19:10:14.065998: Fim do download do arquivo ..\data\01-collected\201408_Compras.zip
2019-09-03 19:10:14.065998: Inicio do download do arquivo ..\data\01-collected\201409_Compras.zip
2019-09-03 19:10:18.504001: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\201409_Compras.zip
2019-09-03 19:10:19.208949: Descompactando o arquivo ..\data\01-collected\201409_Compras.zip
2019-09-03 19:10:19.276767: Fim do download do arquivo ..\data\01-collected\201409_Compras.zip
2019-09-03 19:10:19.276767: Inicio do download do arquivo ..\data\01-collected\201410_Compras.zip
2019-09-03 19:10:23.816801: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\201410_Compras.zip
2019-09-03 19:10:24.526132: Descompactando o arquivo ..\data\01-collected\201410_Co

2019-09-03 19:12:24.415891: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\201605_Compras.zip
2019-09-03 19:12:25.078667: Descompactando o arquivo ..\data\01-collected\201605_Compras.zip
2019-09-03 19:12:25.126455: Fim do download do arquivo ..\data\01-collected\201605_Compras.zip
2019-09-03 19:12:25.127453: Inicio do download do arquivo ..\data\01-collected\201606_Compras.zip
2019-09-03 19:12:28.446777: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\201606_Compras.zip
2019-09-03 19:12:29.112616: Descompactando o arquivo ..\data\01-collected\201606_Compras.zip
2019-09-03 19:12:29.160494: Fim do download do arquivo ..\data\01-collected\201606_Compras.zip
2019-09-03 19:12:29.161486: Inicio do download do arquivo ..\data\01-collected\201607_Compras.zip
2019-09-03 19:12:32.532160: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\201607_Compras.zip
2019-09-03 19:12:33.161884: Descompactando o arquivo ..\data\01-collected\201607_Co

2019-09-03 19:13:55.216638: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\201802_Compras.zip
2019-09-03 19:13:55.866158: Descompactando o arquivo ..\data\01-collected\201802_Compras.zip
2019-09-03 19:13:55.928033: Fim do download do arquivo ..\data\01-collected\201802_Compras.zip
2019-09-03 19:13:55.928989: Inicio do download do arquivo ..\data\01-collected\201803_Compras.zip
2019-09-03 19:13:59.035844: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\201803_Compras.zip
2019-09-03 19:13:59.709088: Descompactando o arquivo ..\data\01-collected\201803_Compras.zip
2019-09-03 19:13:59.758952: Fim do download do arquivo ..\data\01-collected\201803_Compras.zip
2019-09-03 19:13:59.759951: Inicio do download do arquivo ..\data\01-collected\201804_Compras.zip
2019-09-03 19:14:02.704038: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\201804_Compras.zip
2019-09-03 19:14:03.548712: Descompactando o arquivo ..\data\01-collected\201804_Co